# Recoinize

# 1. Setup

## 1.1. Pull code from GitHub

In [5]:
from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
git_update("live-edit", force=True, show_output=False)
# install_requirements()

In [7]:
print_gpu_name()

b'NVIDIA RTX A4000\n'


## 1.4. Import dependencies

In [55]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from keras.models import Sequential, load_model, Model
from datasets import load_dataset

from src.models import make_embedding, make_siamese_model

In [127]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Get data from Hugging Face

In [ ]:
dataset = load_dataset('photonsquid/coins-euro')
dataset = dataset.with_format('tf')
